In [9]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import re

### Clare as an example

In [20]:
url = "https://gaapitchlocator.net/clare"

In [21]:
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'html')

In [22]:
column_names = ["Club", "Latitude","Longitude"]
locations = pd.DataFrame(columns = column_names)
ballyea = soup.find_all('div')
for line in ballyea:    
    if line['class']==['site']: 
        data =line
usefulinfo  = data.find_all('div')
for line in usefulinfo:
    if line['class'] ==['x-main', 'full']:
        clubs = line
clubs = clubs.find_all('div')
i=0
for line in clubs:
    if(line['class'][0]=='lmm-geo-tags'):
        sample = line.text
        locations.loc[i] = [sample.split(":")[0],[float(j) for j in sample.split(":")[1].split(',')][0],[float(j) for j in sample.split(":")[1].split(',')][1]]
        i +=1

In [23]:
locations['Club'] = locations['Club'].str.replace('GAA','').str.replace('Hurling Club','').str.replace(r"\(.*\)","").str.replace("\\", '').str.replace('St','St.').str.replace('GFC','').str.strip()
locations = locations.drop_duplicates(subset=['Club'],keep='last')

In [29]:
url = "https://en.wikipedia.org/wiki/Clare_Senior_Hurling_Championship"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
headings = soup.find_all("h2")

for header in headings:
    if all(x in header.get_text().lower() for x in ['list','final']):
        table =header.find_next_sibling()
    elif all(x in header.get_text().lower() for x in ['roll','honour']):
        table =header.find_next_sibling()
        
right_table=soup.find_all('table', class_='wikitable')
for table in right_table:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings == ['Year', 'Winner', 'Opponent']:
        break
finals = pd.DataFrame(columns = headings)
i=0
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        finals.loc[i] = [th.text.strip() for th in cells]
        i +=1
finals['Winner'] = finals['Winner'].str.extract("([A-Za-z'.,Á-Úá-ú -]+)")
finals['Winner']=finals['Winner'].str.strip()
finals['Opponent'] = finals['Opponent'].str.extract("([A-Za-z'.,Á-Úá-ú -]+)")
finals['Opponent'] = finals['Opponent'].str.strip()
finals['Year'] = finals['Year'].astype(str).str[0:4].astype(int)
finals.set_index('Year',inplace=True)

In [32]:
wikitomap = {"Wolfe Tones":"Wolfe Tones na Sionna","Éire Óg":"Eire Og","Clooney-Quin":"Clooney Quin"}

In [33]:
finals =finals.replace({"Winner":wikitomap,"Opponent":wikitomap})

In [34]:
countytitles19812020 = finals.loc[finals.index > 1980]['Winner'].value_counts()
runnersup19812020 = finals.loc[finals.index > 1980]['Opponent'].value_counts()

In [35]:
locations.set_index('Club',inplace=True)
locations['Countytitles'] = countytitles19812020
locations['Runnersup'] = runnersup19812020 

In [36]:
right_table
for table in right_table:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings == ['Club', 'Last Title']:
        break

In [37]:
seniorteams = pd.DataFrame(columns = headings)
i=0
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==2:
        seniorteams.loc[i] = [th.text.strip() for th in cells]
        i +=1

In [38]:
Intermediate_clubs = ["Smith O'Briens" ,'Corofin',"Tubber","St. Joseph's Doora-Barefield",
                      "Tulla","Bodyke","Ruan","Parteen","Killanena"]
JuniorA_clubs = ["Ogonnelloe","Ennistymon"]
JuniorB_clubs = ["The Banner","Meelick"]
JuniorC_clubs = ["Kilkee Bealatha","Clonboney"]

In [39]:
seniorteams['Club'] = seniorteams['Club'].str.replace(r"\(.*\)","").str.replace(r"\,.*","").str.strip()

In [40]:
seniorteams = seniorteams.replace({"Club":wikitomap})

In [41]:
seniorteams['Senior'] = 1

In [42]:
seniorteams.set_index('Club',inplace=True)

In [43]:
Intermediate_teams = pd.DataFrame(Intermediate_clubs,columns=['Club'])
Intermediate_teams['Intermediate'] = 1 
Intermediate_teams.set_index('Club',inplace=True)

In [44]:
JuniorA_teams = pd.DataFrame(JuniorA_clubs,columns=['Club'])
JuniorA_teams['JuniorA'] = 1 
JuniorA_teams.set_index('Club',inplace=True)
JuniorB_teams = pd.DataFrame(JuniorB_clubs,columns=['Club'])
JuniorB_teams['JuniorB'] = 1 
JuniorB_teams.set_index('Club',inplace=True)
JuniorC_teams = pd.DataFrame(JuniorC_clubs,columns=['Club'])
JuniorC_teams['JuniorC'] = 1 
JuniorC_teams.set_index('Club',inplace=True)

In [45]:
locations['Senior'] = seniorteams['Senior']

In [46]:
locations['intermediate'] = Intermediate_teams['Intermediate']
locations['JuniorA'] = JuniorA_teams['JuniorA']
locations['JuniorB'] = JuniorB_teams['JuniorB']
locations['JuniorC'] = JuniorC_teams['JuniorC']

In [48]:
HurlingClubs = locations[locations.isnull().sum(axis=1)<7].copy()

In [49]:
def level(row):
    if row.loc['Senior'] == 1 :
        return 1
    if row.loc['intermediate']==1:
        return 2
    if row.loc['JuniorA']==1 :
        return 3
    if row.loc['JuniorB'] == 1:
        return 4
    if row.loc['JuniorC']  == 1:
        return 5

In [50]:
levelclub = HurlingClubs.apply(lambda row: level(row), axis=1)
HurlingClubs['level'] = levelclub

In [51]:
HurlingClubs['Countytitles'] = HurlingClubs['Countytitles'].fillna(0).astype(int)

In [52]:
Ouput  = HurlingClubs.loc[:,['Latitude','Longitude','level','Countytitles']]

In [55]:
Ouput.to_csv('Input_data\Scrapped\ClareHurling.csv')

### Clare Hurling vs football

In [56]:
url = "https://en.wikipedia.org/wiki/Clare_Senior_Football_Championship"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
right_table=soup.find_all('table', class_='wikitable')
for table in right_table:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings == ['Year', 'Winner', 'Opponent']:
        break
finals = pd.DataFrame(columns = headings)
i=0
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        finals.loc[i] = [th.text.strip() for th in cells]
        i +=1
finals['Winner'] = finals['Winner'].str.extract("([A-Za-z'.,Á-Úá-ú -]+)")
finals['Winner']=finals['Winner'].str.strip()
finals['Opponent'] = finals['Opponent'].str.extract("([A-Za-z'.,Á-Úá-ú -]+)")
finals['Opponent'] = finals['Opponent'].str.strip()
finals['Year'] = finals['Year'].astype(str).str[0:4].astype(int)
finals.set_index('Year',inplace=True)

In [57]:
countytitles19812020 = finals.loc[finals.index > 1980]['Winner'].value_counts()
runnersup19812020 = finals.loc[finals.index > 1980]['Opponent'].value_counts()
locations['CountyFtitles'] = countytitles19812020
locations['RunnersupF'] = runnersup19812020 

In [58]:
right_table
for table in right_table:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings == ['Club', 'Last Title']:
        break
seniorteams = pd.DataFrame(columns = headings)
i=0
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==2:
        seniorteams.loc[i] = [th.text.strip() for th in cells]
        i +=1

In [59]:
wikitomap = {"Wolfe Tones":"Wolfe Tones na Sionna","Éire Óg":"Eire Og","Clooney-Quin":"Clooney Quin","St. Joseph's, Miltown Malbay":
            "St. Joseph's Miltown Malbay","St. Breckan's":"St. Breckans","Kilmurry-Ibrickane":"Kilmurray Ibrickane"}

In [60]:
seniorteams['Club'] = seniorteams['Club'].str.replace(r"\(.*\)","").str.strip()
seniorteams = seniorteams.replace({"Club":wikitomap})
seniorteams['Club'] = seniorteams['Club'].str.replace(r"\,.*","").str.strip()
seniorteams = seniorteams.replace({"Club":wikitomap})

In [61]:
seniorteams['Senior'] = 1
seniorteams.set_index('Club',inplace=True)


In [62]:
locations['SeniorFootball'] = seniorteams['Senior']

In [63]:
SeniorFootballteams = locations[locations['SeniorFootball']==1]

In [64]:
SeniorFootballteams = SeniorFootballteams[['Latitude','Longitude','SeniorFootball']]

In [68]:
SeniorFootballteams.to_csv('Input_data\Scrapped\ClareSFteams.csv')

In [66]:
SeniorHurlingteams = locations[locations['Senior']==1]
SeniorHurlingteams  = SeniorHurlingteams[['Latitude','Longitude','Senior']]

In [69]:
SeniorHurlingteams.to_csv('Input_data\Scrapped\ClareSHteams.csv')

### All-Ieland wins by county

In [70]:
url = "https://en.wikipedia.org/wiki/All-Ireland_Senior_Hurling_Championship"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
right_table=soup.find_all('table', class_='wikitable')
for table in right_table:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings == ['','Team','Winners','Years won','Runners-up','Years Runners-up']:
        break

In [71]:
allirelands = pd.DataFrame(columns = headings)
i=0
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==6:
        allirelands.loc[i] = [th.text.strip() for th in cells]
        i +=1
    elif len(cells)==5:
        allirelands.loc[i] = [i+1]+[th.text.strip() for th in cells]
        i +=1
    elif len(cells)==4:
        allirelands.loc[i] = [i+1]+[th.text.strip() for th in cells]+[0]
        i +=1

In [72]:
allirelands = allirelands.loc[:,['Team','Winners']]

In [73]:
allirelands = allirelands[allirelands.loc[:,'Team']!='London']

In [74]:
allirelands['Winners'] = allirelands['Winners'].astype(int)

In [75]:
allirelands=allirelands[allirelands['Winners']>0]

In [76]:
allirelands =allirelands[allirelands['Team']!='Kerry'] #There only win in 1891 was by a club rather than the county team 

In [79]:
allirelands.to_csv('Input_data\Scrapped\HurlinAllIreland.csv')

### Hurling Clubs in Hurling counties

In [80]:
column_names = ["Club", "Latitude","Longitude"]
locations = pd.DataFrame(columns = column_names)

In [81]:
i=0
for team in allirelands['Team']:
    url = f"https://gaapitchlocator.net/{team}"
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html')
    ballyea = soup.find_all('div')
    for line in ballyea:    
        if line['class']==['site']:
            data =line
    usefulinfo  = data.find_all('div')
    for line in usefulinfo:
        if line['class'] ==['x-main', 'full']:
            clubs = line
    clubs = clubs.find_all('div')
    for line in clubs:
        if(line['class'][0]=='lmm-geo-tags'):
            sample = line.text
            locations.loc[i] = [f'{team}_'+sample.split(":")[0],[float(j) for j in sample.split(":")[1].split(',')][0],[float(j) for j in sample.split(":")[1].split(',')][1]]
            i +=1

In [82]:
locations['Club'] = locations['Club'].str.replace(' GAA','').str.replace(' Hurling Club','').str.replace(r"\(.*\)","").str.replace("\\", '').str.replace('St ','St. ').str.replace('-',' ').str.replace(' GFC','').str.replace('\'s','s').str.strip()
locations = locations.drop_duplicates(subset=['Club'],keep='last')

In [83]:
i=0
finals = pd.DataFrame(columns = ['Year', 'Winner', 'Opponent'])
county = pd.DataFrame(columns = ['County'])
for team in allirelands['Team']:
    url = f"https://en.wikipedia.org/wiki/{team}_Senior_Hurling_Championship"
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    headings = soup.find_all("h2")
    table = None
    for header in headings:
        if header.get_text().lower() in {'finals listed by year[edit]','list of finals[edit]'}:
            table =header.find_next_sibling()
    if table==None:
        for header in headings:
            if header.get_text().lower() in {'roll of honour[edit]'}:
                table =header.find_next_sibling()
    if len(table.findAll('tr'))<10:
        table = table.find_next_sibling()

    if team in ['Wexford','Cork','Dublin']:
        for row in table.findAll('tr'):
            cells=row.findAll('td')
            if len(cells)!=0:
                info = np.array([th.text.strip() for th in cells])[[0,1,3]]
                if info[1]!='':
                    finals.loc[i] = [info[0],f'{team}_'+info[1],f'{team}_'+info[2]]
                    i +=1 
                    county.loc[i]=team
    else:
        for row in table.findAll('tr'):
            cells=row.findAll('td')
            if len(cells)==3:
                info = np.array([th.text.strip() for th in cells])
                if info[1]!='':
                    finals.loc[i] = [info[0],f'{team}_'+info[1],f'{team}_'+info[2]]
                    i +=1
                    county.loc[i]=team
    
    
    
    
''' right_table=soup.find_all('table', class_='wikitable')
    if team in ['Wexford','Cork','Dublin']:
        for table in right_table:
            ths = table.find_all('th')
            headings = [th.text.strip() for th in ths]
            if (headings[:4] == ['Year','Winners','Score','Runners-up']):
                break 
        for row in table.findAll('tr'):
            cells=row.findAll('td')
            if len(cells)==7:
                info = np.array([th.text.strip() for th in cells])[[0,1,3]]
                finals.loc[i] = [info[0],f'{team}_'+info[1],f'{team}_'+info[2]]
                i +=1
    else:
        for table in right_table:
            ths = table.find_all('th')
            headings = [th.text.strip() for th in ths]
            if (headings == ['Year', 'Winner', 'Opponent'])|(headings ==['Year', 'Winner', 'Runner-Up']):
                break
        for row in table.findAll('tr'):
            cells=row.findAll('td')
            if len(cells)==3:
                info = np.array([th.text.strip() for th in cells])
                finals.loc[i] = [info[0],f'{team}_'+info[1],f'{team}_'+info[2]]
                i +=1
                '''

" right_table=soup.find_all('table', class_='wikitable')\n    if team in ['Wexford','Cork','Dublin']:\n        for table in right_table:\n            ths = table.find_all('th')\n            headings = [th.text.strip() for th in ths]\n            if (headings[:4] == ['Year','Winners','Score','Runners-up']):\n                break \n        for row in table.findAll('tr'):\n            cells=row.findAll('td')\n            if len(cells)==7:\n                info = np.array([th.text.strip() for th in cells])[[0,1,3]]\n                finals.loc[i] = [info[0],f'{team}_'+info[1],f'{team}_'+info[2]]\n                i +=1\n    else:\n        for table in right_table:\n            ths = table.find_all('th')\n            headings = [th.text.strip() for th in ths]\n            if (headings == ['Year', 'Winner', 'Opponent'])|(headings ==['Year', 'Winner', 'Runner-Up']):\n                break\n        for row in table.findAll('tr'):\n            cells=row.findAll('td')\n            if len(cells)==

In [84]:
county['County'].value_counts()

Cork         134
Galway       134
Kilkenny     133
Tipperary    133
Dublin       133
Laois        127
Limerick     126
Clare        125
Offaly       124
Wexford      119
Waterford     86
Name: County, dtype: int64

In [85]:
allirelands['Team']

0      Kilkenny
1          Cork
2     Tipperary
3      Limerick
4        Dublin
5       Wexford
6        Galway
7        Offaly
8         Clare
9     Waterford
11        Laois
Name: Team, dtype: object

In [86]:
finals.loc[270:365]

,Year,Winner,Opponent
270,2017[10],Tipperary_Thurles Sarsfields 1-24,Tipperary_Borris-Ileigh 0-11
271,2016[11],Tipperary_Thurles Sarsfields 0-27,Tipperary_Kiladangan 1-15
272,2015[12],Tipperary_Thurles Sarsfields 1-18,Tipperary_Nenagh Éire Óg 3-11
273,2014[13],Tipperary_Thurles Sarsfields 2-22,Tipperary_Loughmore-Castleiney 3-11
274,2013 [14],Tipperary_Loughmore-Castleiney 1-17,Tipperary_Nenagh Éire Óg 1-16
...,...,...,...
361,1926,Tipperary_Moycarkey-Borris 6-04,Tipperary_Boherlahan 4-02
362,1925,Tipperary_Boherlahan 5-04,Tipperary_Toomevara/Moycarkey-Borris 2-03
363,1924,Tipperary_Boherlahan 8-03,Tipperary_Mid Selection 1-00
364,1923,Tipperary_Toomevara 5-04,Tipperary_Kilmoyler 3-03


In [87]:
allirelands['Team'][~allirelands['Team'].isin(county['County'])]

Series([], Name: Team, dtype: object)

In [88]:
url = f"https://en.wikipedia.org/wiki/Waterford_Senior_Hurling_Championship"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
headings = soup.find_all("h2")
table = None
for header in headings:
    if header.get_text().lower() in {'finals listed by year[edit]','list of finals[edit]'}:
        table =header.find_next_sibling()
if table==None:
    for header in headings:
        if header.get_text().lower() in {'roll of honour[edit]'}:
            table =header.find_next_sibling()
if len(table.findAll('tr'))<10:
    table = table.find_next_sibling()

In [90]:
finals

,Year,Winner,Opponent
0,2020,Kilkenny_Ballyhale Shamrocks 5-19,Kilkenny_Dicksboro 1-10
1,2019,Kilkenny_Ballyhale Shamrocks 2-21,Kilkenny_James Stephens 1-15
2,2018,Kilkenny_Ballyhale Shamrocks 2-20,Kilkenny_Bennettsbridge 2-17
3,2017[24],Kilkenny_Dicksboro 4-15,Kilkenny_James Stephens 4-10
4,2016[25],Kilkenny_O'Loughlin Gaels 0-19,Kilkenny_Ballyhale Shamrocks 1-12
...,...,...,...
1369,1897,Laois_Harristown 0-08,Laois_Rathdowney 1-01
1370,1891,Laois_Clonaslee w/o,Laois_Aghaboe scr
1371,1890,Laois_Clonaslee 1-02,Laois_Rathdowney 1-00
1372,1889,Laois_Rathdowney 3-05,Laois_Skierke 0-00


In [91]:
allirelands['Team']

0      Kilkenny
1          Cork
2     Tipperary
3      Limerick
4        Dublin
5       Wexford
6        Galway
7        Offaly
8         Clare
9     Waterford
11        Laois
Name: Team, dtype: object

In [92]:
finals.loc[400]

Year                              2020
Winner      Limerick_Na Piarsaigh 5-27
Opponent            Limerick_Doon 1-12
Name: 400, dtype: object

In [97]:
Tippwinners = pd.read_csv('Input_data/TippWinners.csv',encoding = "ISO-8859-1")
DublinWinners = pd.read_csv('Input_data/DublinWinners.csv',encoding = "ISO-8859-1")
DublinWinners = DublinWinners[['Year','Winners','Runners-up']]
DublinWinners.columns = ['Year', 'Winner', 'Opponent']
Tippwinners['Winner']= 'Tipperary_'+ Tippwinners['Winner'].astype(str)
Tippwinners['Opponent']= 'Tipperary_'+ Tippwinners['Opponent'].astype(str)
DublinWinners['Winner']= 'Dublin_'+ DublinWinners['Winner'].astype(str)
DublinWinners['Opponent']= 'Dublin_'+ DublinWinners['Opponent'].astype(str)
finals = finals.append(Tippwinners, ignore_index=True)
finals = finals.append(DublinWinners, ignore_index=True)

In [106]:
finals['Year'].replace('', '0000', inplace=True) 

In [107]:
finals['Winner'] = finals['Winner'].str.extract("([A-Za-z _'.,Á-Úá-ú _-]+)")
finals['Winner']=finals['Winner'].str.replace('St ','St. ').str.replace('-',' ').str.replace('\'s','s').str.strip()
finals['Opponent'] = finals['Opponent'].str.extract("([A-Za-z'.,Á-Úá-ú _-]+)")
finals['Opponent'] = finals['Opponent'].str.strip()
finals['Year'] = finals['Year'].astype(str).str[0:4].astype(int)
finals.set_index('Year',inplace=True)

In [108]:
countytitles19812020 = finals.loc[finals.index > 1980]['Winner'].value_counts()
runnersup19812020 = finals.loc[finals.index > 1980]['Opponent'].value_counts()

In [109]:
countytitles19812020.index.values[[i not in locations['Club'].values for i in countytitles19812020.index.values]]

array(['Wexford_Oulart the Ballagh', 'Offaly_Birr', 'Galway_Athenry',
       'Wexford_Rathnure', 'Galway_St. Thomass', 'Cork_Imokilly',
       'Cork_Blackrock', 'Offaly_Kilcormac', 'Laois_Clough',
       'Tipperary_Kiladangan', 'Tipperary_Mullinahone',
       'Tipperary_Nenagh Éire Óg', 'Cork_Glen Rovers',
       'Tipperary_Kilruane MacDonaghs', 'Clare_Wolfe Tones',
       'Laois_Borris in Ossory', 'Kilkenny_Young Irelands',
       'Clare_Éire Óg, Ennis', 'Cork_Avondhu', 'Cork_Carbery',
       'Wexford_St. Annes', 'Wexford_Naomh Éanna'], dtype=object)

In [110]:
WikitoMap = {'Wexford_Oulart the Ballagh':'Wexford_Oulart The Ballagh', "Offaly_Birr":"Offaly_St. Brendans Park",
            "Wexford_Rathnure":'Wexford_Rathnure St. Annes',"Galway_Athenry":'Galway_St. Marys Athenry',
            "Galway_St. Thomass":"Galway_St. Thomas","Cork_Imokilly":'Cork_Midleton',"Offaly_Kilcormac":'Offaly_Kilcormac Killoughey',
            "Laois_Clough":"Laois_Clough Ballacolla",
            "Kilkenny_Young Irelands":'Kilkenny_Young Irelands Gowran', "Clare_Éire Óg, Ennis":"Clare_Eire Og", "Clare_Wolfe Tones":'Clare_Wolfe Tones na Sionna',
            "Waterford_Passage":'Waterford_Passage East',"Tipperary_Kilruane MacDonaghs":'Tipperary_Kilruane McDonaghs',
            "Tipperary_Nenagh Éire Óg":'Tipperary_Eire Og Nenagh',"Tipperary_Kiladangan":'Tipperary_Kildangan','Cork_Carbery':'Cork_Carbery Rangers',
            "Cork_Avondhu":'Cork_Charleville Rovers','Wexford_Naomh Éanna':'Wexford_Naomh Eanna Gorey',
            "Tipperary_Mullinahone":"Tipperary_Mullinahone CJ Kickhams","Laois_Borris in Ossory":'Laois_Borris In Ossory',
            'Cork_Blackrock':'Cork_Blackrock National','Cork_Glen Rovers':'Cork_Glen Rovers Hurling'
            }

In [111]:
countytitles19812020 = countytitles19812020.rename(index=WikitoMap)
runnersup19812020 = runnersup19812020.rename(index=WikitoMap)

In [112]:
locations.set_index('Club',inplace=True)

In [113]:
countytitles19812020 =  countytitles19812020.groupby(countytitles19812020.index).first().copy()

In [114]:
locations['Countytitles'] = countytitles19812020

In [115]:
CountyWinners = locations[np.isnan(locations['Countytitles'])==False]

In [118]:
CountyWinners.to_csv('Input_data\Scrapped\CountyWinners.csv')